# 1. Introduction et contexte

## 1.1 Présentation du problème

L’ADEME (Agence de l’Environnement et de la Maîtrise de l’Energie) a récemment lancé un appel à manifestation d’intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité pour les personnes et les marchandises adaptées à différents types de territoires.

Votre structure CesiCDP est déjà bien implantée dans le domaine. Aidé de nombreux partenaires, vous avez réalisé plusieurs études sur le thème de la Mobilité Multimodale Intelligente. Les nouvelles technologies de transport, plus économiques et moins polluantes ne sont pas sans poser de nouveaux défis notamment d’un point de vue de l’optimisation de la gestion des ressources. Mais ces problèmes de logistique du transport présentent un enjeu majeur pour l’avenir : ces applications sont nombreuses (distribution du courrier, livraison de produits, traitement du réseau routier, ramassage des ordures) et leur impact sur l’environnement peut être véritablement significatif.

Vous faites partie de l’équipe (4 personnes) mise en place par CesiCDP pour répondre à l’appel de l’ADEME. L’enjeu est d’obtenir de nouveaux marchés avec des financements très intéressants pour continuer à développer votre activité.

CesiCDP a décidé d’orienter son étude sur la gestion de tournées de livraison. Le problème algorithmique consiste à calculer sur un réseau routier une tournée permettant de relier entre elles un sous-ensemble de villes, puis de revenir à son point de départ, de manière à minimiser la durée totale de la tournée. Cette optimisation devra tenir compte du trafic prévu sur chaque axe pour les différentes tranches horaires.

L’idée est de proposer une méthode issue de la Recherche Opérationnelle pour générer une tournée de livraison correspondant à ce problème.

Le périmètre reste encore à préciser. Vous avez décrit une version de base du problème. Mais, afin de le rendre plus réaliste et retenir toute l’attention de l’ADEME, vous hésitez à ajouter des contraintes supplémentaires. Il faut s’attendre à ce qu’il soit ainsi plus dur à traiter.

# 2. Définition formelle du problème

## 2.1 Modélisation linéaire

Dans la recherche opérationnelle, la programmation linéaire permet de modéliser un problème de décision en utilisant des variables et des contraintes.

Pour l'utiliser, il y a plusieurs notions à définir :
- les variables de décision : ce sont les choix à effectuer ; dans notre cas, ça peut être les arêtes que l'on va prendre pour faire le plus court chemin.
- la fonction économique : se définit par une minimisation / maximisation (minimisation d'un coût ou la maximisation d'un bénéfice)
- les contraintes : définissent l'espace de solution possible.

### 2.1.1 Variables de décision

Pour les variables de décision, on a décidé d'utiliser des variables binaires :

$$x_{ij} \in \{0, 1\} \quad \forall (i,j) \in \{1, \dots, n\},  i \neq j$$

Ici, $n$ est le nombre de villes (taille du graphe).

Ce qui veut dire que :
- $x_{ij} = 1$ si le véhicule emprunte l'arc de $i$ vers $j$
- $x_{ij} = 0$ sinon

On doit également représenter le coût de passage des villes :

$$c_{ij} \quad \forall (i,j) \in \{1, \dots, n\},  i \neq j $$

Où $c_{ij}$ représente le coût pour passer de la ville $i$ à la ville $j$.

On peut enfin noter $G$ notre graphe représentant la ville.

Mais aussi : $V$ qui représente l'ensemble des sommets

Et enfin : $E$ qui représente l'ensemble des arêtes

### 2.1.2 Variables auxiliaires

On doit également définir des variables auxiliaires. Elles vont permettre l'élimination des sous-tours :

$$u_i \in \mathbb{R}^+ \quad \forall i \in V \setminus \{0\}$$

- $u_i$ : position de la ville $i$ dans la cycle (formulation MTZ - Miller-Tucker-Zemlin)

### 2.1.3 Fonction économique (ou objectif)

L'objectif est de minimiser la durée totale de la tournée :

$$\min = \sum_{i=1} \sum_{j=1, j \neq i} x_{ij} \cdot c_{ij}$$

Interprétation : on fait la somme les temps de trajet de toutes les arêtes empruntés. Si l'arête est emprunté, alors son coût est ajouté à la somme.

### 2.1.4 Contraintes

Le véhicule part exactement une fois du dépôt :

$$\sum_{j \in V \setminus \{0\}} x_{0j} = 1$$

Le véhicule revient exactement une fois au dépôt :

$$\sum_{i \in V \setminus \{0\}} x_{i0} = 1$$

Chaque ville (hors dépôt) est visitée exactement une fois (on entre et on sort une fois) :

$$\sum_{i \in V \setminus \{j\}} x_{ij} = 1 \quad \forall j \in V \setminus \{0\}$$

Ces contraintes empêchent la formation de cycles disjoints (sous-tournées, MTZ) :

$$u_i - u_j + n \cdot x_{ij} \leq n-1 \quad \forall i,j \in V \setminus \{0\}, i \neq j$$

## 2.2 Contraintes supplémentaires

<span style="color:red ; font-size:20px">Nous avons choisi 3 contraintes un peu plus car nous voulons pouvoir palier a des difficultés sur une d'elle :</span>

<span style="color:green ; font-size:18px">Fenêtres temporelles (version de base)
</span>

Permet d’intégrer la contrainte horaire des clients (heures d’ouverture/fermeture). Apporte du réalisme sans complexité excessive. Modélisation standard du VRPTW (Vehicle Routing Problem with Time Windows).


<span style="color:green ; font-size:18px">Trafic dynamique (version avancée)
</span>

Cœur du sujet ADEME : modélise le trafic urbain et périurbain selon l’heure. Ajoute une dimension temporelle réaliste (congestion matin/soir) et différencie les trajets selon les tranches horaires. C’est la contrainte avancée, comptant pour deux.


<span style="color:green ; font-size:18px">Flotte hétérogène
(version de base)</span>

Sert de structure simple et stable pour le modèle. Tous les véhicules ont les mêmes caractéristiques, ce qui permet de se concentrer sur les contraintes principales (trafic + temps). Réduit la complexité algorithmique tout en restant cohérent.


<span style="color:red ; font-size:20px">Maintenant voyons pourquoi nous avons pas choisi les autres contraintes :</span>

<span style="color:green ; font-size:18px">Flotte hétérogène (version avancée)</span>

Complexifie fortement la gestion du temps (le véhicule peut arriver avant et attendre). Oblige à recalculer dynamiquement les horaires pour tous les clients → surcharge algorithmique pour un gain limité.

<span style="color:green ; font-size:18px">Flotte hétérogène (version avancée)</span>

Introduit un sous-problème de bin-packing 3D (chargement). Besoin de modéliser dimensions, compatibilités et règles logistiques. Très réaliste industriellement, mais hors du périmètre du projet ADEME.

<span style="color:green ; font-size:18px">Points de collecte
(version avancée)</span>

Transforme le problème en MDVRP, à deux niveaux (affectation dépôt–client + routage). Complexité trop élevée, demande une méta-heuristique multi-niveau. Trop long pour un projet de démonstration.

<span style="color:green ; font-size:18px">Trafic dynamique, Points de collecte (version de base)</span>

Trop simple et pas cohérent avec l’objectif ADEME. N’apporte aucun réalisme temporel.

# 3. Étude de la complexité

## 3.1 Classe de complexité

## 3.2 Complexité temporelle